In [1]:
import nltk

In [7]:
from nltk.corpus import treebank_chunk

In [17]:
sentence="US unveils world's most powerful supercomputer, beats China"

In [18]:
data=treebank_chunk.chunked_sents()

In [19]:
traindata=data[:3500]

In [20]:
testdata=data[3500:]

In [21]:
print(traindata[7])

(S
  (NP A/DT Lorillard/NNP spokewoman/NN)
  said/VBD
  ,/,
  ``/``
  (NP This/DT)
  is/VBZ
  (NP an/DT old/JJ story/NN)
  ./.)


In [22]:
from nltk.chunk import RegexpParser

In [23]:
tagged_simple_sent=nltk.pos_tag(nltk.word_tokenize(sentence))

In [24]:
print('POS Tags:', tagged_simple_sent)

POS Tags: [('US', 'NNP'), ('unveils', 'JJ'), ('world', 'NN'), ("'s", 'POS'), ('most', 'RBS'), ('powerful', 'JJ'), ('supercomputer', 'NN'), (',', ','), ('beats', 'NNS'), ('China', 'NNP')]


In [25]:
chunk_grammar="""NP:{<DT>?<JJ>*<NN.*>}"""

In [26]:
rc=RegexpParser(chunk_grammar)

In [27]:
c=rc.parse(tagged_simple_sent)

In [28]:
print(c)

(S
  (NP US/NNP)
  (NP unveils/JJ world/NN)
  's/POS
  most/RBS
  (NP powerful/JJ supercomputer/NN)
  ,/,
  (NP beats/NNS)
  (NP China/NNP))


In [29]:
grammar="""NP:{<DT>?<JJ>?<NN.*>}
           ADJP:{<JJ>}
           ADVP:{<RB.*>}
           PP:{<IN>}
           VP:{<MD>?<VB.*>+}

"""

In [30]:
rc=RegexpParser(grammar)

In [31]:
c=rc.parse(tagged_simple_sent)

In [33]:
print(c)

(S
  (NP US/NNP)
  (NP unveils/JJ world/NN)
  's/POS
  (ADVP most/RBS)
  (NP powerful/JJ supercomputer/NN)
  ,/,
  (NP beats/NNS)
  (NP China/NNP))


In [34]:
print(rc.evaluate(testdata))

ChunkParse score:
    IOB Accuracy:  46.1%%
    Precision:     19.9%%
    Recall:        43.3%%
    F-Measure:     27.3%%


In [35]:
from nltk.chunk.util import tree2conlltags, conlltags2tree

In [36]:
train_sent=traindata[7]

In [37]:
print(train_sent)

(S
  (NP A/DT Lorillard/NNP spokewoman/NN)
  said/VBD
  ,/,
  ``/``
  (NP This/DT)
  is/VBZ
  (NP an/DT old/JJ story/NN)
  ./.)


In [38]:
wtc=tree2conlltags(train_sent)

In [39]:
wtc

[('A', 'DT', 'B-NP'),
 ('Lorillard', 'NNP', 'I-NP'),
 ('spokewoman', 'NN', 'I-NP'),
 ('said', 'VBD', 'O'),
 (',', ',', 'O'),
 ('``', '``', 'O'),
 ('This', 'DT', 'B-NP'),
 ('is', 'VBZ', 'O'),
 ('an', 'DT', 'B-NP'),
 ('old', 'JJ', 'I-NP'),
 ('story', 'NN', 'I-NP'),
 ('.', '.', 'O')]

In [40]:
tree=conlltags2tree(wtc)

In [41]:
print(tree)

(S
  (NP A/DT Lorillard/NNP spokewoman/NN)
  said/VBD
  ,/,
  ``/``
  (NP This/DT)
  is/VBZ
  (NP an/DT old/JJ story/NN)
  ./.)


In [42]:
def conll_tag_chunks(chunk_sents):
    tagged_sents=[tree2conlltags(tree) for tree in chunk_sents]
    return [[(t,c) for (w,t,c) in sent] for sent in tagged_sents]

In [43]:
def combined_tagger(train_data,taggers,backoff=None):
    for tagger in taggers:
        backoff=tagger(train_data, backoff=backoff)
        return backoff